In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

In [ ]:
!ls -lha kaggle.json

In [ ]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d tapakah68/segmentation-full-body-mads-dataset

In [ ]:
!unzip /content/segmentation-full-body-mads-dataset.zip

In [ ]:
img_path = "/content/segmentation_full_body_mads_dataset_1192_img/images"
mask_path = "/content/segmentation_full_body_mads_dataset_1192_img/masks"
import tensorflow as tf
import numpy as np
import os
import glob

In [ ]:
img_list=[]
for root, dirs, files in os.walk(os.path.abspath(img_path)):
    for file in files:
        img_list.append(os.path.join(root, file))
        

mask_list=[]
for root, dirs, files in os.walk(os.path.abspath(mask_path)):
    for file in files:
        mask_list.append(os.path.join(root, file))

img_list.sort()
mask_list.sort()

len(img_list)

In [ ]:
def _parse_function(filename,label):
    image_string = tf.io.read_file(filename, "file_reader")
    image_decoded = tf.io.decode_png(image_string, channels=3)
    tf.image.random_flip_left_right(image_decoded, seed=1)
    image = tf.cast(image_decoded, tf.float32)/255
    mask_string = tf.io.read_file(label, "file_reader")
    mask_decoded = tf.io.decode_png(mask_string, channels=1)
    tf.image.random_flip_left_right(mask_decoded, seed=1)
    mask = tf.cast(mask_decoded, tf.float32)/255
    return image,mask

def data_augment(image, mask):
    image = tf.image.random_hue(image, 0.01)
    image = tf.image.random_saturation(image, 0.70, 1.30)
    image = tf.image.random_contrast(image, 0.80, 1.20)
    image = tf.image.random_brightness(image, 0.10)
    return image, mask


dataset = tf.data.Dataset.from_tensor_slices((tf.constant(img_list), tf.constant(mask_list)))
dataset = dataset.map(_parse_function)
dataset = dataset.map(data_augment)



dataset = dataset.batch(32,drop_remainder=True)

AUTOTUNE = tf.data.AUTOTUNE

def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(256)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

dataset = configure_for_performance(dataset)


len_ds = len(dataset)
len_ts = int(len(dataset)*0.8)
len_vs = int(len_ds - len_ts)

train_ds = dataset.take(len_ts)
val_ds = dataset.skip(len_ts).take(len_vs)
print(len(dataset),len(train_ds),len(val_ds))


In [ ]:
for image, mask in val_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Mask shape: ", mask.numpy().shape)


In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, LeakyReLU, BatchNormalization, MaxPool2D,Conv2DTranspose, concatenate,Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Lambda,Dropout

inputs = tf.keras.layers.Input((384, 512, 3))
s = tf.keras.layers.Lambda(lambda x: x / 1)(inputs)

#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer=tf.keras.optimizers.Adamax, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.{epoch:02d}-{val_loss:.2f}.hdf5", 
                              monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq="epoch")

In [ ]:
model.fit(train_ds,epochs=25,validation_data=val_ds,verbose=1,callbacks=[checkpoint])

In [ ]:
img = tf.keras.preprocessing.image.load_img("/content/segmentation_full_body_mads_dataset_1192_img/images/HipHop_HipHop1_C0_00180.png")
input_array = tf.keras.preprocessing.image.img_to_array(img)
input_array = np.array([input_array])  # Convert single image to a batch.
predictions = model.predict(input_array)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.imshow(tf.keras.preprocessing.image.array_to_img(np.squeeze(predictions)[:, :, np.newaxis]))